In [ ]:
elastic_rods_dir = '../../elastic_rods/python/'
weaving_dir = '../'
import os, sys; sys.path.append(elastic_rods_dir); sys.path.append(weaving_dir)
import numpy as np, elastic_rods, linkage_vis
from bending_validation import suppress_stdout as so
from matplotlib import pyplot as plt
import importlib

# Heart Coarse 1
default_camera_parameters = ((3.466009282140468, -4.674139805388271, -2.556131049738206), (-0.21402574298422497, -0.06407538766530313, -0.9747681088523519),(0.1111, 0.1865, 0.5316))
RIBBON_CS = [1/150, 1/15]
MODEL_NAME = "heart_coarse_1"
MODEL_PATH = weaving_dir + f'scaled_objs/models/{MODEL_NAME}.obj'
SUBDIVISION_RESOLUTION = 20
INPUT_SURFACE_PATH = weaving_dir + f'scaled_objs/surface_models/{MODEL_NAME}.obj'
RIBBON_NAME = "heart_coarse_1_strip"

In [ ]:
# Bird
RIBBON_CS = [0.1/600, 0.1/60]
MODEL_NAME = "bird_close_beak_0.1"
MODEL_PATH = weaving_dir + f'models/{MODEL_NAME}.obj'
SUBDIVISION_RESOLUTION = 20
SMOOTHING_WEIGHT = 10
REGULARIZATION_WEIGHT = 0
INPUT_SURFACE_PATH = weaving_dir + 'surface_models/bird_0.1.obj'
RIBBON_NAME = "bird_close_beak_0.1"

In [ ]:
linkage = elastic_rods.SurfaceAttractedLinkage(INPUT_SURFACE_PATH, useCenterline=True, linkage_path=MODEL_PATH,
                                               subdivision=SUBDIVISION_RESOLUTION, initConsistentAngle=False,
                                               rod_interleaving_type=elastic_rods.InterleavingType.triaxialWeave)
linkage.setMaterial(elastic_rods.RodMaterial('rectangle', 2000, 0.3, RIBBON_CS, stiffAxis=elastic_rods.StiffAxis.D1))
linkage.set_holdClosestPointsFixed(True);
linkage.set_attraction_tgt_joint_weight(0.01);
linkage.attraction_weight = 100;

In [ ]:
linkage.updateRotationParametrizations()
linkage.updateSourceFrame()
with so(): elastic_rods.compute_equilibrium(linkage)

In [ ]:
lview = linkage_vis.LinkageViewer(linkage)
lview.show()

In [ ]:
# Validate CrossingForceObjective against Python implementation of crossing force analysis
import structural_analysis
print(0.5 * np.linalg.norm(np.clip(structural_analysis.weavingCrossingForceMagnitudes(linkage)[:, 0], 0.0, None))**2)
print(0.5 * np.linalg.norm(structural_analysis.weavingCrossingForceMagnitudes(linkage)[:, 1])**2)

import linkage_optimization
cfo = linkage_optimization.ContactForceObjective(linkage)

cfo.normalWeight =  cfo.boundaryNormalWeight = 1.0
cfo.tangentialWeight = cfo.boundaryTangentialWeight = 0.0
print(cfo.value())

cfo.normalWeight =  cfo.boundaryNormalWeight = 0.0
cfo.tangentialWeight = cfo.boundaryTangentialWeight = 1.0
print(cfo.value())

In [ ]:
currDoFs = linkage.getExtendedDoFsPSRL()
save_tgt_joint_pos = linkage.jointPositions();
WOE = linkage_optimization.WeavingOptEnergyType

In [ ]:
import finite_diff
importlib.reload(finite_diff)
fdw = finite_diff.DesignOptimizationTermFDWrapper(cfo, linkage)

linkage.updateSourceFrame()
linkage.updateRotationParametrizations()
d = np.random.uniform(-1, 1, linkage.numExtendedDoFPSRL())

### Contact Force Gradient Validation

In [ ]:
cfo.normalWeight = 1.0
cfo.boundaryNormalWeight = 0.25
cfo.tangentialWeight = cfo.boundaryTangentialWeight = 0.0
cfo.normalActivationThreshold = -0.1
fdw.setDoFs(currDoFs)
linkage.updateSourceFrame()
finite_diff.gradient_convergence_plot(fdw, direction=d)

In [ ]:
fdw.setDoFs(currDoFs)
linkage.updateSourceFrame()
cfo.normalWeight = cfo.boundaryNormalWeight = 0.0
cfo.tangentialWeight = 1.0
cfo.boundaryTangentialWeight = 0.5
finite_diff.gradient_convergence_plot(fdw, direction=d)

### Contact Force Hess-vec Validation

Normal contact force term is non-smooth due to the clamp to zero, so the finite differences do not converge nicely...

In [ ]:
# Try to perturb away from discontinuity
#linkage.setExtendedDoFsPSRL(currDoFs + 2e-3 * np.random.uniform(-1, 1, linkage.numExtendedDoFPSRL()))
#linkage.updateSourceFrame()
fdw.setDoFs(currDoFs)
cfo.normalWeight = 1.0
cfo.boundaryNormalWeight = 0.25
cfo.tangentialWeight = cfo.boundaryTangentialWeight = 0.0
cfo.normalActivationThreshold = -0.1 # Choosing a low activation threshold also avoids the discontinuity
finite_diff.hessian_convergence_plot(fdw, direction=d)

But the tangential term is well-behaved.

In [ ]:
fdw.setDoFs(currDoFs)
#linkage.setExtendedDoFsPSRL(currDoFs + 2e-3 * np.random.uniform(-1, 1, linkage.numExtendedDoFPSRL()))
#linkage.updateSourceFrame()
cfo.normalWeight = cfo.boundaryNormalWeight = 0.0
cfo.tangentialWeight = 1.0
cfo.boundaryTangentialWeight = 0.5
finite_diff.hessian_convergence_plot(fdw, direction=d)

# Target fitting validation

In [ ]:
linkage.set_holdClosestPointsFixed(True)
linkage.set_attraction_tgt_joint_weight(0.01)
linkage.attraction_weight = 0.1

In [ ]:
OPTS = elastic_rods.NewtonOptimizerOptions()
OPTS.gradTol = 1e-6
OPTS.niter = 200
OPTS.beta = 1e-8
RIBBON_NAME = "sphere_strip"

In [ ]:
linkage.set_design_parameter_config(use_restLen = True, use_restKappa = True)
useCenterline = True
optimizer = linkage_optimization.WeavingOptimization(linkage, INPUT_SURFACE_PATH, useCenterline, equilibrium_options=OPTS, pinJoint = 0, useFixedJoint = False)
optimizer.set_target_joint_position(save_tgt_joint_pos)

In [ ]:
tf = linkage_optimization.TargetFittingDOOT(linkage, optimizer.target_surface_fitter)

In [ ]:
tf.value()

In [ ]:
currDoFs = linkage.getExtendedDoFsPSRL()

In [ ]:
importlib.reload(finite_diff)
fdw_tf = finite_diff.DesignOptimizationTermFDWrapper(tf, linkage)

### Closest Points Fixed

In [ ]:
optimizer.set_holdClosestPointsFixed(True)

In [ ]:
fdw_tf.setDoFs(currDoFs)
finite_diff.gradient_convergence_plot(fdw_tf, direction=d)

In [ ]:
fdw_tf.setDoFs(currDoFs)
finite_diff.hessian_convergence_plot(fdw_tf, direction=d)

### Closest Points Freed

In [ ]:
optimizer.set_holdClosestPointsFixed(False)

In [ ]:
fdw_tf.setDoFs(currDoFs)
finite_diff.gradient_convergence_plot(fdw_tf, direction=d)

In [ ]:
fdw_tf.setDoFs(currDoFs)
finite_diff.hessian_convergence_plot(fdw_tf, direction=d)

### Investigate non-differentiability
Try to verify it's due to closest points jumping around

In [ ]:
def findBadSensitivities(eps, customBadPts = None):
    tsf = optimizer.target_surface_fitter
    fdw_tf.setDoFs(currDoFs + eps * d)
    cpPlus = tsf.linkage_closest_surf_pts.copy().reshape((-1, 3))
    cpiPlus = np.array(optimizer.target_surface_fitter.linkage_closest_surf_tris)
    fdw_tf.setDoFs(currDoFs - eps * d)
    cpMinus = tsf.linkage_closest_surf_pts.copy().reshape((-1, 3))
    cpiMinus = np.array(optimizer.target_surface_fitter.linkage_closest_surf_tris)

    fdw_tf.setDoFs(currDoFs)
    N = optimizer.target_surface_fitter.N[tsf.linkage_closest_surf_tris, :]
    normalVelocity = ((N @ (cpPlus - cpMinus).T).diagonal() / (2 * eps))
    print(np.linalg.norm(normalVelocity))
    if (customBadPts is not None):
        badPts = customBadPts
    else:
        badPts = np.where(np.abs(normalVelocity) > 1e-6)[0]
    print(normalVelocity[badPts])
    
    cp = tsf.linkage_closest_surf_pts.copy().reshape((-1, 3)) 
    fdw_tf.setDoFs(currDoFs)
    return badPts, cpMinus[badPts], cpPlus[badPts]

In [ ]:
findBadSensitivities(1e-6)

In [ ]:
findBadSensitivities(2e-6)

In [ ]:
# Indeed, we have a closest point jump substantially under slightly larger steps.
# Apparently this is due to a query point almost on the medial axis. 

In [ ]:
_, cpp, cpm = findBadSensitivities(1.57375e-6, customBadPts=3782)
np.linalg.norm(cpp - cpm)

In [ ]:
_, cpp, cpm = findBadSensitivities(1.57e-6, customBadPts=3782)
np.linalg.norm(cpp - cpm)

# Regularization term wrapper validation

In [ ]:
rcs = linkage_optimization.RestCurvatureSmoothingDOOT(linkage)

In [ ]:
fdw_rcs = finite_diff.DesignOptimizationTermFDWrapper(rcs, linkage)

In [ ]:
fdw_rcs.setDoFs(currDoFs + 1e-6 * np.random.uniform(-1, 1, fdw_rcs.numDoF())) # perturb out of straight configuration
finite_diff.gradient_convergence_plot(fdw_rcs, direction=d)

In [ ]:
fdw_rcs.setDoFs(currDoFs)
finite_diff.hessian_convergence_plot(fdw_rcs, direction=d)

# Composite objective validation

In [ ]:
doo = linkage_optimization.DesignOptimizationObjective()

In [ ]:
rlm = linkage_optimization.RestLengthMinimizationDOOT(linkage)
eeo = linkage_optimization.ElasticEnergyObjective(linkage)

In [ ]:
doo.add([('CrossingForce',           WOE.Full,           cfo),
         ('SmoothingRegularization', WOE.Smoothing,      rcs),
         ('RestLengthMinimization',  WOE.Regularization, rlm),
         ('ElasticEnergy',           WOE.Elastic,        eeo),
         ('TargetFitting',           WOE.Target,         tf)])

In [ ]:
doo.value()

In [ ]:
doo.values()

In [ ]:
for t in doo.terms: t.term.weight = 0

In [ ]:
for t in doo.terms: t.term.weight = 1

In [ ]:
doo.terms

In [ ]:
fdw_doo = finite_diff.DesignOptimizationTermFDWrapper(doo, linkage)

In [ ]:
for tr in doo.terms:
    print(tr.name, np.linalg.norm(tr.term.computeGrad() - tr.term.grad()))

In [ ]:
doo.terms[0].term.weight = 1.0

In [ ]:
finite_diff.gradient_convergence_plot(fdw_doo, direction=d)

In [ ]:
fdw_doo.setDoFs(currDoFs)
finite_diff.hessian_convergence_plot(fdw_doo, direction=d)

In [ ]:
#doo.terms.clear()

# WeavingOptimization Validation

In [ ]:
import fd_weaver_editor
OPTS = elastic_rods.NewtonOptimizerOptions()
OPTS.gradTol = 1e-6
OPTS.verbose = 10;
OPTS.beta = 1e-8
OPTS.niter = 200
OPTS.verboseNonPosDef = False

linkage.set_design_parameter_config(use_restLen = True, use_restKappa = True)
useCenterline = True
wopt = linkage_optimization.WeavingOptimization(linkage, INPUT_SURFACE_PATH, useCenterline, equilibrium_options=OPTS, useFixedJoint = False)
wopt.set_target_joint_position(save_tgt_joint_pos)

wopt.rl_regularization_weight = 0
wopt.smoothing_weight = 10
wopt.beta = 500000.0
wopt.gamma = 1

In [ ]:
wopt.objective.terms[-1].term.value()

In [ ]:
wopt.J(currDoFs[-wopt.numParams():])

In [ ]:
wopt.objective.terms[-1].term.weight = 1e6

In [ ]:
wopt.objective.terms[-1].term.value()

In [ ]:
wopt.objective.terms[-1].term.tangentialWeight = 0.0
wopt.objective.terms[-1].term.normalWeight = 1.0

In [ ]:
wopt.objective.values()

In [ ]:
wopt.objective.terms

In [ ]:
fd_weaver_editor.gradient_convergence_plot(wopt, d[-wopt.numParams():], 'Full', WOE.Full)

In [ ]:
#linkage_optimization.benchmark_reset()
fd_weaver_editor.hessian_convergence_plot(wopt, d[-wopt.numParams():], 'Full', WOE.Full)
#linkage_optimization.benchmark_report()